# Загрузка Pandas и очистка данных

In [83]:
import pandas as pd
import re
from IPython.display import clear_output
from copy import copy
from datetime import datetime, timedelta, date
from IPython.display import display, clear_output

In [2]:
df_read = pd.read_csv('main_task.csv')

In [3]:
# Ваш код по очистке данных и генерации новых признаков
# При необходимости добавьте ячейки

### Первичное приведение данных в порядок

In [3]:
## Удаление столбцов: ID ресторана, URL страница, ID в базе TripAdvisor
df_read.drop(['Restaurant_id','URL_TA','ID_TA'], axis=1, inplace=True)

In [4]:
## Приведение диапазона цен к более удобному виду: ['$','$$','$$$']
df_read['Price Range'] = df_read['Price Range'].replace('$$ - $$$','$$').replace('$$$$','$$$')

In [5]:
## Столбец с кухнями приводим к списку и добавляем столбец с количеством кухонь
def count_cuisine(row):
    if not pd.isna(row['Cuisine Style']):
        row['Cuisine Style'] = re.sub(r'[\'\[\]]', '', row['Cuisine Style']).split(', ')
        row['Count cuisine'] = len(row['Cuisine Style'])
    return row

df_read = df_read.apply(lambda row: count_cuisine(row), axis = 1)

In [6]:
## Добавляем столбец принодлежности к столице
def capital(row):
    cap_list = ['Paris','Stockholm','London','Berlin','Bratislava','Vienna','Rome','Madrid',
                   'Dublin','Brussels','Warsaw','Budapest','Copenhagen','Amsterdam','Lisbon',
                   'Prague','Oslo','Helsinki','Ljubljana','Athens','Luxembourg','Edinburgh']
    cap_list = set(cap_list)
    if row['City'] in cap_list:
        row['Capital'] = 1
    else:
        row['Capital'] = 0
    return row

df_read = df_read.apply(lambda row: capital(row), axis = 1)


In [7]:
## Делаем столбец с разницей дней между комментариями
def diff_day(row):
    diff = re.sub(r'[\'\[\]]', '',row['Reviews'].split('], [')[1]).split(', ')
    if len(diff) == 2:
        day1 = datetime.strptime(diff[1], '%m/%d/%Y')
        day2 = datetime.strptime(diff[0], '%m/%d/%Y')
        row['Diff day'] = abs((day2 - day1).days)
    else:
        row['Diff day'] = None
    return row

df_read = df_read.apply(lambda row: diff_day(row), axis = 1)

### Добавление столбцов соответствующих категориальным признакам

In [8]:
## Добавление столбцов соответствующих городам
df = pd.concat([df_read,pd.get_dummies(df_read['City'])],axis=1)

In [9]:
## Удаляем столбец с названием городов
df.drop('City',axis=1,inplace=True)

In [10]:
## Добавление столбцов соответствующих стоимости ресторана
df = pd.concat([df,pd.get_dummies(df_read['Price Range'])],axis=1)

In [11]:
## Удаляем столбец со стоимостью ресторана
df.drop('Price Range',axis=1,inplace=True)

In [12]:
## Делаем список со всеми кухнями
df['Cuisine Style'].fillna(0,inplace=True)
cuisine_list = set()
for i, rec in df.iterrows():
    if rec['Cuisine Style'] != 0:
        for cuisine in rec['Cuisine Style']:
            cuisine_list.add(cuisine)

In [13]:
## Заполняем столбцы с кухнями нашими значениями
def cuis_b(row):
    if row['Cuisine Style'] != 0:
        for i in row['Cuisine Style']:
            row[i] = 1
    return row

for cuisine in cuisine_list:
    df[cuisine] = 0
    
df = df.apply(lambda row: cuis_b(row), axis = 1)


In [67]:
df_test = df.drop(['Cuisine Style','Reviews'], axis=1)

In [39]:
## Вариант заполнения значений медианными
df_test['Count cuisine'].fillna(3,inplace=True)
df_test['Number of Reviews'].fillna(33,inplace=True)
df_test['Diff day'].fillna(67,inplace=True)

In [68]:
## Вариант "простого" заполнения значений
df_test['Count cuisine'].fillna(1,inplace=True)
df_test['Number of Reviews'].fillna(0,inplace=True)
df_test['Diff day'].fillna(0,inplace=True)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [69]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df_test.drop(['Rating'], axis = 1)
y = df_test['Rating']

In [70]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [71]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [72]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [73]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [74]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.2130415


In [ ]:
Стартовое значение - MAE: 0.431341367063492

### Проверка средних результатов модели

In [90]:
summ = 0
for i in range(50):
    df_test = df.drop(['Cuisine Style','Reviews'], axis=1)
    df_test.dropna(inplace=True)
    
    X = df_test.drop(['Rating'], axis = 1)
    y = df_test['Rating']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    
    regr = RandomForestRegressor(n_estimators=100)
    regr.fit(X_train, y_train)
    y_pred = regr.predict(X_test)
    
    summ += metrics.mean_absolute_error(y_test, y_pred)
    
    clear_output()
    print(49 - i)

0


При удалении       - MAE: 0.21173425 (среднее 50 прогонов)

Медианные значения - MAE: 0.21381119 (среднее 50 прогонов)

Значения [1,0,0]   - MAE: 0.21351588 (среднее 50 прогонов)

In [92]:
summ / 50

0.21173424679487185